In [30]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 904 kB 3.4 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [67]:
import json
import requests
from selenium_driver import setProxy
import time
import re
from selenium.webdriver.common.action_chains import ActionChains

In [95]:
link_descuentos = 'https://www.mercadolibre.com.ar/ofertas?cat=#origin=scut&filter_position=1&is_recommended_domain=false'
driver = setProxy(headless=True)

In [23]:
actions = ActionChains(driver)

In [16]:
driver.get(link_descuentos)

In [17]:
prom_class = 'promotion-item__link-container'

In [18]:
actions = ActionChains(driver)

In [43]:
cant_paginas = 209
with open('discount_items_links2.txt','a') as fi:
    for pag in range(4,210):
        driver.get('https://www.mercadolibre.com.ar/ofertas?page={}'.format(pag))
        for elem in driver.find_elements_by_class_name(prom_class):
            fi.write(elem.get_attribute('href')+'\n')

        
        time.sleep(2)

In [37]:
driver.get('https://www.mercadolibre.com.ar/ofertas?page=2')

In [44]:
with open('discount_items_links2.txt','r') as fi:
    selenium_items_links = fi.readlines()

In [91]:
driver.get(selenium_items_links[199])

In [94]:
driver.close()

In [83]:
selenium_items_links[199]

'https://articulo.mercadolibre.com.ar/MLA-883990644-detergente-para-lavavajillas-finish-automatico-powerball-all-in-1-max-tabletas-repuesto-30u-_JM?hide_psmb=true#reco_item_pos=7&reco_backend=promotions-sorted-by-score-mla-A&reco_backend_type=low_level&reco_client=seller-promotions&reco_id=993271fa-c505-41da-bcf9-e09e6cbfb359&deal_print_id=5e7ff7a0-2833-11eb-a08e-eb65606e9454&model_version=recommendations/prueba-promotions-MLA__3628__KAKUY&promotion_type=TODAY_PROMOTION\n'

In [92]:
(driver.find_element_by_class_name('ui-pdp-review__amount').text).split()[0]

'49'

In [97]:
clase_descripcion = 'ui-pdp-description__content'

with open('discount_items_rating_descrip.txt','a') as fi:
    fail_count = 0
    for item_link in selenium_items_links[2:]:
        driver.get(item_link)
        item_id = 'MLA'+re.search(r'.*\/MLA-(\d+)-.*',item_link).group(1)
        try:
            fail_count = 0
            rewiew_amount = (driver.find_element_by_class_name('ui-pdp-review__amount').text).split()[0]
            parent_rating = driver.find_element_by_class_name('ui-pdp-review__ratings')
            rating_stars_ls = [[x.get_attribute('class').split('-')[-1],x.get_attribute('width')] for x in parent_rating.find_elements_by_xpath("./*")]
            stars = 0
            for stars_ls in rating_stars_ls:
                if stars_ls[0] == 'full' and stars_ls[1] == '10': # Representa una estrella completa, si el segundo elemento es 11 la estrella esta vacia
                    stars += 1
                elif stars_ls[0] == 'half': # Representa una media estrella
                    stars += 0.5
                else:
                    pass
            
        except:
            stars = 0
            review_amount = '0'
            fail_count += 1
        try:
            descripcion = (driver.find_element_by_class_name(clase_descripcion).text).replace('\n',' ')
        except:
            descripcion = 'None'
        fi.write('_\t\t_'.join([item_id,str(stars),rewiew_amount,descripcion])+'\n')
        
        if fail_count >= 100:
            print('Todo mal :c')
            break

In [48]:
selenium_items_links[1]

'https://articulo.mercadolibre.com.ar/MLA-693504981-cama-elastica-precio-oferta-3-metros-ninos-adulto-grande-_JM?variation=28515514448&hide_psmb=true#reco_item_pos=1&reco_backend=promotions-sorted-by-score-mla-A&reco_backend_type=low_level&reco_client=seller-promotions&reco_id=d98ff632-ded7-4c1f-9678-db1bbb6d520e&deal_print_id=f26c6d00-2832-11eb-b20e-81231f32d6fd&model_version=recommendations/prueba-promotions-MLA__3628__KAKUY&promotion_type=DEAL_OF_THE_DAY\n'

In [ ]:
for link in selenium_items_links:
    driver.get(link)
    

In [20]:
json_dict = []
with open('items_test.jl','r') as fi:
    for line in fi.readlines():
        json_dict.append(json.loads(line))

In [22]:
len(json_dict)

count = 0
for item in json_dict:
    if item['original_price'] != None:
        count += 1
print(count)

In [3]:
item1 = 'MLA873113583'

In [4]:
response = requests.get('https://api.mercadolibre.com/items?ids={}'.format(item1))

In [5]:
json.loads(response.text)

[{'code': 200,
  'body': {'id': 'MLA873113583',
   'site_id': 'MLA',
   'title': 'Tostadora Kanji Home Kjh-tm140 1400w Metal 4 Panes 6 Niveles',
   'subtitle': None,
   'seller_id': 12626860,
   'category_id': 'MLA10068',
   'official_store_id': None,
   'price': 2899,
   'base_price': 2899,
   'original_price': 3399,
   'currency_id': 'ARS',
   'initial_quantity': 81,
   'available_quantity': 50,
   'sold_quantity': 5,
   'sale_terms': [{'id': 'WARRANTY_TIME',
     'name': 'Tiempo de garantía',
     'value_id': None,
     'value_name': '6 meses',
     'value_struct': {'number': 6, 'unit': 'meses'},
     'values': [{'id': None,
       'name': '6 meses',
       'struct': {'number': 6, 'unit': 'meses'}}]},
    {'id': 'WARRANTY_TYPE',
     'name': 'Tipo de garantía',
     'value_id': '2230279',
     'value_name': 'Garantía de fábrica',
     'value_struct': None,
     'values': [{'id': '2230279',
       'name': 'Garantía de fábrica',
       'struct': None}]}],
   'buying_mode': 'buy_it_now